# Cleaning & Transformation Script for the NYC Citywide Annualized Calendar Sales Data

**Link To Data**: https://data.cityofnewyork.us/City-Government/NYC-Citywide-Annualized-Calendar-Sales-Update/w2pb-icbu

**API end-point**: https://data.cityofnewyork.us/resource/w2pb-icbu.json

**Data Dictionary**: https://data.cityofnewyork.us/api/views/w2pb-icbu/files/8ed811b4-8238-4b5e-9acc-1e33d8705498?download=true&filename=Annualized_Calendar_Sales_Update%20Data_Dictionary.xlsx

**Cleaned Data Dictionary**: https://docs.google.com/spreadsheets/d/17XyGmnw2fZuTMCWVKB1XiWGHQuwqWOidm0w80lbIyjE/edit?usp=sharing

**IMPORTANT: This data set is 121.3 MB. Once downloaded, please keep the file in the same directory as this jupyter notebook file, so that the .csv file can be read into a pandas DataFrame correctly. After downloading the dataset, please rename it to 'NYC_sales.csv', to avoid any errors in reading the file**

- First, we will read the data into a pandas DataFrame and start analyzing the data to gather insights on what should be cleaned
- Afterwards, we can start rearranging columns, renaming as needed, and removing NULL/Duplicate values if necessary
- After cleaning the data, we can then transform the data by creating new FACT columns generated. These columns can then be analyzed later for actionable insights


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Reset warning filter to default (optional)
warnings.filterwarnings("default")

In [ ]:
import pandas as pd

# Reads CSV file into a pandas DataFrame:
df = pd.read_csv('NYC_sales.csv')

In [ ]:
# Checks the first five records of the DataFrame:
df.head(5)

In [ ]:
# Checks the number of rows and columns in the DataFrame
df.shape

In [ ]:
# Checks the column names in the DataFrame
df.columns

In [ ]:
# Checks the data types of each column in the DataFrame
df.dtypes

In [ ]:
# Drop columns from columns_to_drop list and into a new DataFrame called df_dropped
# That way you do not overwrite the original DataFrame from above
columns_to_drop = ['BUILDING CLASS CATEGORY','BLOCK','LOT','EASE-MENT','APARTMENT NUMBER','Longitude','Latitude','Community Board', 'Council District','Census Tract','BBL','Census Tract 2020','NTA','NTA Code']
df_dropped = df.drop(columns=columns_to_drop)

# Display the new DataFrame after dropping columns
print("\nDataFrame after dropping columns:")
df_dropped.head(5)

In [ ]:
# Rearrange columns
column_order = ['BIN','SALE DATE', 'SALE PRICE','ADDRESS', 'BOROUGH', 'NEIGHBORHOOD','ZIP CODE', 'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS', 'LAND SQUARE FEET', 'GROSS SQUARE FEET','BUILDING CLASS AT TIME OF SALE','BUILDING CLASS AS OF FINAL ROLL','TAX CLASS AT TIME OF SALE','TAX CLASS AS OF FINAL ROLL','YEAR BUILT']
df_dropped = df_dropped[column_order]

# Display the DataFrame after rearranging columns
print("\nDataFrame after rearranging columns:")
df_dropped.head(5)

### Now, let's count the number of Null/NaN records and Duplicate records. This will help us to understand areas we need to clean from our DataFrame. 
**For example, Since 'BIN' is our unique identifier, we will want to count the number of null/NaN values there in order to understand how many records we will need to drop later on in our transformation phase.**

In [ ]:
# Count NaN records in column 'BIN'
nan_count = df_dropped['BIN'].isna().sum()
# Display the count of NaN records
print("\nNumber of NaN records in column 'BIN':", nan_count)

In [ ]:
# Count NaN records in each column of the DataFrame
nan_counts = df_dropped.isna().sum()

# Display the count of NaN records for each column of the DataFrame
print("\nNumber of NaN records in each column:")
print(nan_counts)

In [ ]:
# Count duplicate records in the entire DataFrame
duplicate_count = df_dropped.duplicated().sum()

# Display the count of duplicate records
print("\nNumber of duplicate records in the DataFrame:", duplicate_count)

# Transformation Deliverables:
**1. Unified date format YYYY-MM-DD**

In [ ]:
# Convert the 'Date' column to a unified date format (YYYY-MM-DD)
df_dropped['SALE DATE'] = pd.to_datetime(df_dropped['SALE DATE'], errors='coerce').dt.strftime('%Y-%m-%d')

# Display the DataFrame after converting the date format
print("\nDataFrame after converting the SALE DATE column to a unified date format (YYYY-MM-DD):")
df_dropped.head(5)

**2.  Splitting the date into multiple units (Year, Month, Day)**

In [ ]:
# Convert the 'SALE DATE' column to datetime
df_dropped['SALE DATE'] = pd.to_datetime(df_dropped['SALE DATE'], errors='coerce')

# Extract Year, Month, and Day into separate columns
df_dropped['YEAR_SOLD'] = df_dropped['SALE DATE'].dt.year
df_dropped['MONTH_SOLD'] = df_dropped['SALE DATE'].dt.month
df_dropped['DAY_SOLD'] = df_dropped['SALE DATE'].dt.day

# Display the DataFrame after splitting the date into multiple units
print("\nDataFrame after splitting the SALE DATE:")
df_dropped.head(5)

In [ ]:
# Rename columns
new_column_names = {'SALE DATE': 'SALE_DATE', 'SALE PRICE': 'SALE_PRICE', 'ZIP CODE': 'ZIP_CODE', 'RESIDENTIAL UNITS': 'RESIDENTIAL_UNITS', 'COMMERCIAL UNITS': 'COMMERCIAL_UNITS', 'TOTAL UNITS': 'TOTAL_UNITS', 'LAND SQUARE FEET': 'LAND_SQFT', 'GROSS SQUARE FEET': 'GROSS_SQFT', 'BUILDING CLASS AT TIME OF SALE': 'INITIAL_BUILDING_CLASS', 'BUILDING CLASS AS OF FINAL ROLL': 'FINAL_BUILDING_CLASS', 'TAX CLASS AT TIME OF SALE': 'INITIAL_TAX_CLASS', 'TAX CLASS AS OF FINAL ROLL': 'FINAL_TAX_CLASS', 'YEAR BUILT':'YEAR_BUILT' }
df_dropped.rename(columns=new_column_names, inplace=True)

# Drop duplicate columns
#columns_to_drop = ['YEAR','MONTH','DAY']
#df_dropped = df_dropped.drop(columns=columns_to_drop)

# Display the DataFrame after renaming columns
print("\nDataFrame after renaming columns:")
df_dropped.head(5)

**3. Removing NULL/NaN values**
- This step is important to do before we start changing data types
- For example, if we want to change a column's data type from float to int, but there is a record with a null/NaN value, you will get an error: **IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer**
- So, it's best to get rid of these null values first, to avoid any errors later

In [ ]:
# Count the number of rows and columns
df_dropped.shape

#Drop null/NaN values from each row in the DataFrame that has atleast 1 null/NaN value
df_cleaned = df_dropped.dropna()

# Count the number of rows and columns after DataFrame has been cleaned
df_cleaned.shape

**4. Removing Duplicate rows**

In [ ]:
# Count the number of rows, columns in the cleaned DataFrame
df_cleaned.shape

# Drop duplicate rows in the cleaned DataFrame
df_cleaned.drop_duplicates(inplace=True)

#Count the number of rows, columns in the cleaned DataFrame after dropping duplicates
df_cleaned.shape

**5. Verify Data against data reference**

In [ ]:
# Convert appropiate datatypes to int as necessary
df_cleaned['BIN'] = df_cleaned['BIN'].astype(int)
df_cleaned['ZIP_CODE'] = df_cleaned['ZIP_CODE'].astype(int)
df_cleaned['RESIDENTIAL_UNITS'] = df_cleaned['RESIDENTIAL_UNITS'].astype(int)
df_cleaned['COMMERCIAL_UNITS'] = df_cleaned['COMMERCIAL_UNITS'].astype(int)
df_cleaned['TOTAL_UNITS'] = df_cleaned['TOTAL_UNITS'].astype(int)
df_cleaned['YEAR_BUILT'] = df_cleaned['YEAR_BUILT'].astype(int)

# Get rid of ',' and '-' in the 'LAND_SQFT' and 'GROSS_SQFT' columns
df_cleaned['LAND_SQFT'] = df_cleaned['LAND_SQFT'].str.replace(',', '')
df_cleaned['LAND_SQFT'] = df_cleaned['LAND_SQFT'].str.replace('-', '')
df_cleaned['GROSS_SQFT'] = df_cleaned['GROSS_SQFT'].str.replace(',', '')
df_cleaned['GROSS_SQFT'] = df_cleaned['GROSS_SQFT'].str.replace('-', '')

# Convert the 'FINAL_TAX_CLASS' to numeric with errors='coerce'
# errors='coerce' converts the column to numeric values, otherwise non-convertible values are replaced with NaN
df_cleaned['FINAL_TAX_CLASS'] = pd.to_numeric(df_cleaned['FINAL_TAX_CLASS'], errors='coerce')

# Convert the remaining column to numeric
df_cleaned['LAND_SQFT'] = pd.to_numeric(df_cleaned['LAND_SQFT'])
df_cleaned['GROSS_SQFT'] = pd.to_numeric(df_cleaned['GROSS_SQFT'])
df_cleaned['FINAL_TAX_CLASS'] = pd.to_numeric(df_cleaned['FINAL_TAX_CLASS'])

# Convert 'INITIAL_TAX_CLASS' column into float data type to match FINAL_TAX_CLASS column data type
df_cleaned['INITIAL_TAX_CLASS'] = df_cleaned['INITIAL_TAX_CLASS'].astype(float)

In [ ]:
# Display DataFrame with edited data types
df_cleaned.head(5)

In [ ]:
# Check to see if datatypes have been converted sucessfully
df_cleaned.dtypes

**Here, we are cleaning the data further. Since, the 'BIN' column has duplicate records, we must remove those records since 'BIN' is our unique identifier column**

In [ ]:
# Count the number of duplicate values in the 'BIN' column
duplicate_count = df_cleaned.duplicated(subset=['BIN']).sum()

# Display the count of duplicate values
print("\nNumber of duplicate values in 'BIN':", duplicate_count)

# Drop records with the same value in 'BIN'
# keep='first' will keep the first record and delete it's duplicates
df_cleaned = df_cleaned.drop_duplicates(subset=['BIN'], keep='first')

# Count the number of duplicate values in the 'BIN' column again
duplicate_count = df_cleaned.duplicated(subset=['BIN']).sum()

# Display the count of duplicate values
print("\nNumber of duplicate values in 'BIN' after cleaning:", duplicate_count)

# See the new shape of the cleaned DataFrame
df_cleaned.shape

**Now, you should have a near-fully cleaned data set based on your requirements. We can then move on to using our FACT columns to create aggregable columns that can provide us with actionable insights**

**6. Adding one or many columns**

In [ ]:
# Add column(s) for properties that have not been sold
df_cleaned['PROPERTIES_UNSOLD'] = df_cleaned['SALE_PRICE'] <= 0
df_cleaned['PROPERTIES_UNSOLD_PRE_2020'] = (df_cleaned['YEAR_SOLD'].isin([2017, 2018, 2019])) & (df_cleaned['SALE_PRICE'] <= 0)
df_cleaned['PROPERTIES_UNSOLD_POST_2020'] = (df_cleaned['YEAR_SOLD'].isin([2020, 2021, 2022])) & (df_cleaned['SALE_PRICE'] <= 0)

# Add column(s) for properties that have been sold
df_cleaned['PROPERTIES_SOLD_POST_2020'] = (df_cleaned['YEAR_SOLD'].isin([2020, 2021, 2022])) & (df_cleaned['SALE_PRICE'] > 0)
df_cleaned['PROPERTIES_SOLD_PRE_2020'] = (df_cleaned['YEAR_SOLD'].isin([2017, 2018, 2019])) & (df_cleaned['SALE_PRICE'] > 0)

# Show DataFrame with new column
df_cleaned.head(5)

# Count Number of properties not yet sold
count_properties_unsold = df_cleaned['PROPERTIES_UNSOLD'].sum()
print(f'Number of properties not yet sold: {count_properties_unsold}')

# Count Number of properties not yet sold pre-pandemic
count_properties_unsold_pre_2020 = df_cleaned['PROPERTIES_UNSOLD_PRE_2020'].sum()
print(f'Number of properties not yet sold pre-pandemic: {count_properties_unsold_pre_2020}')

# Count Number of properties not yet sold post-pandemic
count_properties_unsold_post_2020 = df_cleaned['PROPERTIES_UNSOLD_POST_2020'].sum()
print(f'Number of properties not yet sold post-pandemic: {count_properties_unsold_post_2020}')

# Count Number of properties sold post-pandemic
count_properties_sold_post_2020 = df_cleaned['PROPERTIES_SOLD_POST_2020'].sum()
print(f'\nNumber of properties sold post-pandemic: {count_properties_sold_post_2020}')

# Count Number of properties sold pre-pandemic
count_properties_sold_pre_2020 = df_cleaned['PROPERTIES_SOLD_PRE_2020'].sum()
print(f'Number of properties sold pre-pandemic: {count_properties_sold_pre_2020}')

# Find the difference of properties un-sold pre and post pandemic
difference_properties_unsold_preandpost_2020 = count_properties_unsold_post_2020 - count_properties_unsold_pre_2020
print(f'\nDifference in properties unsold pre and post pandemic: {difference_properties_unsold_preandpost_2020} more properties unsold post-pandemic')
print('Conclusion: There are more properties unsold post-pandemic.')

# Find the difference of properties sold pre and post pandemic
difference_properties_sold_preandpost_2020 = count_properties_sold_pre_2020 - count_properties_sold_post_2020
print(f'\nDifference in properties sold pre and post pandemic: {difference_properties_sold_preandpost_2020} more properties sold pre-pandemic')
print('Conclusion: There are more properties sold pre-pandemic.')

**Below, you can run the following codes to find unique values in columns. You can then gather more insights with those unique values.**

**Some ideas are:**
- Find the years with the most sold properties
- Find the months with the most sold properties
- Find the days with the most sold properties
- Find the average number of sales in each month or year
- Average number of residential units in properties sold

**You can also find which months had more active sales pre and post 2020 to see how the pandemic may have affected sales. Please note that the pandemic started in March 2020. The months before may or may not reflect the effects of the pandemic yet.**

In [ ]:
# Find unique values in columns
unique_years = df_cleaned['YEAR_SOLD'].unique()
unique_years = sorted(unique_years)

unique_months = df_cleaned['MONTH_SOLD'].unique()
unique_months = sorted(unique_months)

unique_days = df_cleaned['DAY_SOLD'].unique()
unique_days = sorted(unique_days)

print(unique_years)
print(unique_months)
print(unique_days)